In [199]:
#importing modules

import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.ticker as mtick
import time
from matplotlib import pyplot as plt
from datetime import date, datetime, timedelta
import re 
from re import search
import pyodbc as pc
pd.set_option('display.max_columns', None)
import calendar
from time import strptime
import os

In [200]:
#variables
today = date.today()
todaysdate = datetime.now()
currentMonth = datetime.now().month

In [201]:
#setting up sql connection and importing data
conn_bi = pc.connect('''Driver={SQL Server};
                            Server=sqlbi;
                            Database=csn_junk;
                            Trusted_Connection=yes'''
                         )

#sourcedata

shipmentlistsql = 'select * from csn_reporting_bi.dbo.tblflexiblebillingspolist'
oceanratesql = 'select * from csn_reporting_bi.dbo.OceanRates'
bafsql = 'select * from csn_reporting_bi.dbo.BR_Monthly_BAF_Rates'
drayratesql = 'select * from csn_reporting_bi.dbo.DrayRates'
currencyconversionsql = 'select * from csn_reporting_bi.dbo.BR_Currency_Conversion'
alreadybilledsql = 'select * from csn_reporting_bi.dbo.BRChargesSpoLevel' 


#converting source data to df
shipmentinput = pd.read_sql(shipmentlistsql, conn_bi)
oceanrate = pd.read_sql(oceanratesql, conn_bi)
drayrate = pd.read_sql(drayratesql, conn_bi)
baf = pd.read_sql(bafsql, conn_bi)
currencyconversion = pd.read_sql(currencyconversionsql, conn_bi)
alreadygenerated = pd.read_sql(alreadybilledsql, conn_bi)

#closing connection
conn_bi.close() 

In [202]:
##combining baf & currency conversion tables with shipmentlist

shipmentlist = shipmentinput

shipmentlist['EffectiveGateIn'] = shipmentlist['EffectiveGateIn'].astype('datetime64')
shipmentlist['MonthTxt'] = shipmentlist['EffectiveGateIn'].dt.month.apply(lambda x: calendar.month_abbr[x])


#add additional months here
baf = baf.rename(columns={'october':'Oct', 'september':'Sep','November':'Nov','december':'Dec',
                          'january':'Jan','february':'Feb','march':'Mar','april':'Apr','origin':'BafOrigin', 'destination':'BafDestination'})

bafmelt = pd.melt(baf, id_vars=['SignedMonth', 'BafOrigin','BafDestination'], 
               var_name = 'MonthTxt', value_name='BafRate' ).dropna().reset_index(drop=True)

### Need to merge on signed month as well, once this is in db
shipmentlist = pd.merge(shipmentlist,bafmelt,how='left', on=['BafOrigin', 'BafDestination', 'MonthTxt', 'SignedMonth'])
shipmentlist = pd.merge(shipmentlist,currencyconversion, on=['ShipmentID','ServiceLevel'],how='left' )

In [203]:
#finding the destination country
shipmentlist['country'] = shipmentlist['DestinationPort'].str[0:2]

def definecountry(s):
    if s['country'] == 'US':
        return 'USA'
    if s['country'] == 'CA':
        return 'CAN'
    if s['country'] == 'GB':
        return 'GB'
    else:
        return 'DE'
    
shipmentlist['country'] = shipmentlist.apply(definecountry, axis=1)

In [204]:
#Writing in Exceptions

#Exceptions: Port Codes - fixing bad port Codes in source data
origincodefix = {

#Syntax: 'BadPortCode':'CorrectPortCode'
    'CNSHA':'CNSHG',
    'CNXMN':'CNXMG',
    'CNSGH':'CNSHG',
    'VNBHA':'VNSGN',
    'VNDNA':'CNSHG',
    'VNVUT':'VNSGN',
    'VNBDU':'VNSGN',
    'KHSCH':'KHKOS',
    'CNNGB':'CNNBG',
    'CNNKGP':'CNNJG',
    'CNTAO':'CNQDG',
    'VNQNT':'VNUIH',
}

destinationcodefix = {
    'USORF':'USCVG',
    'USNWK':'USNYC',
    'USFWT':'USDAL',
    'USLTX':'USDAL',
    'USPER':'USLGB',
    'DEHMY':'DEHAM',    
}

shipmentlist['OriginPort'] = shipmentlist['OriginPort'].map(origincodefix).fillna(shipmentlist['OriginPort'])
shipmentlist['DestinationPort'] = shipmentlist['DestinationPort'].map(destinationcodefix).fillna(shipmentlist['DestinationPort'])

#Exception: Kassel - All DE shipments are going to Hammersbach but cube shows they went to Kassel 
shipmentlist['whid'].replace(to_replace=33191, value=172055, inplace=True)

#Exception: LLY Tech has a unique AAS rate into USHOU, for the rest of shipments however the portcode needs to be USDAL  
shipmentlist['DestinationPort'] = np.where(((shipmentlist['DestinationPort'] == 'USHOU') & ((shipmentlist['SuID'] != 10909) 
                                    & (shipmentlist['ServiceLevel'] != 'OND'))),'USDAL',shipmentlist['DestinationPort'])

In [205]:
#formatting data

oceanrate['SuID'] = oceanrate['SuID'].astype('int')
drayrate['SuID'] = drayrate['SuID'].astype('int')
shipmentlist['ContainerSize'] =shipmentlist['ContainerSize'].astype('int')

#converting dates
dateconvert = ['ATD', 'GateOut', '1stReceipt']
shipmentlist[dateconvert] = shipmentlist[dateconvert].astype('datetime64')

#dropping extra effective + expiration dates
try: 
    oceanrate = oceanrate.drop(columns = ['ScExpiredDate', 'ScEffectiveDate'])
    drayrate = drayrate.drop(columns = ['ScExpiredDate', 'ScEffectiveDate','DestinationPort'])
    
except:
    print('Already Removed')
    
drayrate = drayrate.drop_duplicates().reset_index(drop=True)
drayrate = drayrate[(drayrate['DrayRate'] > 0)]

In [206]:
#adding ocean and dray rates to each shipment

shipmentswocean = pd.merge(shipmentlist,oceanrate, 
                           on=('OceanContract', 'SuID', 'DestinationPort', 'OriginPort', 'ServiceLevel','ContainerSize'),
                           how ='left')

spostobill = pd.merge(shipmentswocean,drayrate[['DrayContract','SuID','whid', 'ServiceLevel', 'DrayRate','ContainerSize']],
                     on=('DrayContract','SuID','whid','ServiceLevel','ContainerSize'), how ='left')  

rates = ['OceanRate', 'DDOCRate', 'DrayRate', 'SecurityFee', 'DTHC', 'BafRate', 'PSS Rate']
spostobill[rates] = spostobill[rates].apply(pd.to_numeric)

In [207]:
#Modifying rates + adding standard rates

#Exception: standard dray rates
spostobill['DrayRate'] = np.where((spostobill['country'] == 'GB') & (spostobill['ServiceLevel'] == 'ACN')
                                  ,10.4,spostobill['DrayRate'])
spostobill['DrayRate'] = np.where((spostobill['country'] == 'DE') & (spostobill['ServiceLevel'] == 'ACN')
                                  ,10.9,spostobill['DrayRate'])
spostobill['DrayRate'] = np.where((spostobill['country'] == 'CAN') & (spostobill['ServiceLevel'] == 'ACN')
                                  ,8,spostobill['DrayRate'])
spostobill['DrayRate'] = np.where((spostobill['whid'] == 33194)& (spostobill['ServiceLevel'] == 'NVO')
                                  ,566,spostobill['DrayRate'])

In [208]:
#supplier exceptions

#Exception: GFW
gfwdrop = spostobill[(spostobill['SuID'] ==  13955) & (spostobill['ATD'].dt.month  !=  (currentMonth-1))] 
spostobill = spostobill.drop(gfwdrop.index, axis=0)

#Exception: TGC 
spostobill['SuID'] = np.where(spostobill['SuID'] == 13758,65140,spostobill['SuID']) 

In [209]:
#billing functions

def pierpass(df): 
    if df['ServiceLevel'] == 'ACN' and df['DestinationPort'] == 'USLGB':
        return df['CBM'] * 1.03
    if df['ServiceLevel'] == 'ACN' and df['DestinationPort'] == 'USLAX':
        return df['CBM'] * 1.03
    if df['ServiceLevel'] == 'ACN' and df['DestinationPort'] == 'USOAK':
        return df['CBM'] * .56
    elif df['DestinationPort'] == 'USOAK': 
        return df['CBM'] * 36.4
    elif df['DestinationPort'] == 'USLGB':
        return df['CBM'] * 66.95
    elif df['DestinationPort'] == 'USLAX':
        return df['CBM'] * 66.95
    else:
        return 0

def caliport (df):
    if df['DestinationPort'] in ['USLGB', 'USOAK']:
        return 'yes'
    else:
        return 'no'
    
    
def drayfuelbill (df):
    if df['country'] == 'USA' and df['cali'] == 'yes':
        return df['California Percent'] * df['draybasecharge']
    if df['country'] == 'USA' and df['cali'] == 'no':
        return df['US Percent'] * df['draybasecharge']
    else:
        return 0
    
def bunkerrate(df):
    if df['ServiceLevel'] == 'DRA':
        return 0
    elif df['ServiceLevel'] == 'ACN':
        return df['CBM'] / 65 * df['BafRate']
    else:
        return df['CBM'] * df['BafRate']

    
cranwhid = [13221, 13217, 156348]
def cranfee(df): 
    if df['ServiceLevel'] == 'ACN' and df['whid'] in [13221, 13217, 156348]:
        return df['CBM'] * .2615
    elif df['whid'] in [13221, 13217, 156348]:
        return 17
    else:
        return 0

In [210]:
#generating charges
spostobill['cali'] = spostobill.apply(caliport, axis=1)

#AC
spostobill['cfscharge'] = np.where((spostobill['ServiceLevel'] == 'ACN'), spostobill['CBM']*10, 0) * spostobill['CFS Exchange Rate']

#oceancharges
spostobill['oceanbasecharge'] = spostobill['OceanRate'] * spostobill['CBM'] * spostobill['Ocean Exchange Rate']
spostobill['bunker'] = spostobill.apply(bunkerrate, axis=1) * spostobill['Ocean Exchange Rate']
spostobill['ddoccharge'] =  spostobill['DDOCRate'] * spostobill['Ocean Exchange Rate'] * spostobill['CBM']
spostobill['psscharge'] = spostobill['CBM'] * spostobill['PSS Rate'] * spostobill['Ocean Exchange Rate']
spostobill['pierpasscharge'] = np.where(spostobill['ServiceLevel'] != 'DRA',spostobill.apply(pierpass, axis=1),0) * spostobill['Ocean Exchange Rate']
spostobill['Securitycharge'] =  np.where(spostobill['country'].isin(['DE','GB']),spostobill['SecurityFee'],0) * spostobill['Ocean Exchange Rate']
spostobill['DTHCFee'] =  np.where(spostobill['country'].isin(['DE','GB']),spostobill['DTHC'],0) * spostobill['Ocean Exchange Rate']
spostobill['CranFee'] = spostobill.apply(cranfee, axis=1)

#draycharges
spostobill['draybasecharge'] = spostobill['DrayRate'] * spostobill['CBM'] * spostobill['Dray Exchange Rate']
spostobill['drayfuelcharge'] = np.where(spostobill['ServiceLevel'].isin(['ACN','NVO','OND', 'DRA','NOR']),
                                        spostobill['draybasecharge']*spostobill['DFSCPercentage'],0) * spostobill['Dray Exchange Rate']
#CargoProtect Charges
cargoprotectrates = ['cfscharge', 'oceanbasecharge', 'ddoccharge'
                     ,'bunker','psscharge', 'draybasecharge', 'drayfuelcharge','Securitycharge', 'DTHCFee']
spostobill['InsuranceCharge'] = (spostobill[cargoprotectrates].sum(axis=1) + spostobill['Wholesale Value'])/100*spostobill['CargoProtectRate']

#adding all charges up
passthroughs = ['pierpasscharge', 'CranFee']
spostobill['TotalAmount'] = spostobill[cargoprotectrates].sum(axis=1) + spostobill['InsuranceCharge'] + spostobill[passthroughs].sum(axis=1) 

In [211]:
#Formatting and saving to T-Drive  Q/A file 


#narrowing columns
allcharges = spostobill[['ShipmentID','OceanContract','DrayContract','ACISpoID', 'SuID','Supplier Currency', 'ServiceLevel',
                         'SignedMonth', 'CBM', 'ContainerNumber', 'EffectiveGateIn','GateIn', 'ATD','1stReceipt','GateOut',
                         'OriginPort', 'DestinationPort','DestinationWarehouse','country','whid', 'wmswhid','cfscharge', 
                         'oceanbasecharge', 'ddoccharge','bunker','pierpasscharge', 'psscharge','draybasecharge', 
                         'drayfuelcharge','Securitycharge','DTHCFee', 'InsuranceCharge', 'CranFee','TotalAmount','CFS Exchange Rate', 
                         'Ocean Exchange Rate','Dray Exchange Rate']]


#fixing service levels desigination and breaking out OaaS suppliers
allcharges['ServiceLevel'] = np.where(allcharges['ServiceLevel'] == 'NOR','NVO',allcharges['ServiceLevel'])
allcharges['ServiceLevel'] = np.where(((allcharges['SuID'].isin([36651, 12128, 405])) 
                                       & (allcharges['ServiceLevel'] == 'OND')),'OaaS',allcharges['ServiceLevel'])


#formatting
finalrates = cargoprotectrates + ['InsuranceCharge', 'TotalAmount'] + passthroughs
allcharges[finalrates] = allcharges[finalrates].round(2)

#comma in Oaas warehouse names messed up CSV
allcharges['DestinationWarehouse'] = allcharges['DestinationWarehouse'].str.replace(',','')

#removing spos that are fully billed
chargestoaddtosql = pd.merge(allcharges,alreadygenerated[['Fully Billed', 'ShipmentID', 'Some Charges']],
                             on = 'ShipmentID', how='left') 
shipmentstobill = chargestoaddtosql[(chargestoaddtosql['Fully Billed'] == 'No')]

#dropping any duplicates
allcharges = allcharges.drop_duplicates().reset_index(drop=True)
shipmentstobill = shipmentstobill.drop_duplicates().reset_index(drop=True)

C:\Users\rl409b\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rl409b\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\rl409b\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [212]:
#Begin converting file from output Q/A file to bulk upload file 

#narrowing shipments based on charge generation trigger & eliminating null GateIns except for DRA Only Shipments
billingtrigger = today#.replace(day=1) - timedelta(days=5)
notdra = shipmentstobill[(shipmentstobill['ATD'] < billingtrigger) & (shipmentstobill['ServiceLevel'] != 'DRA')] 

notdra = notdra[(notdra['GateIn'].notnull())]
dra = shipmentstobill[(shipmentstobill['GateOut'] < billingtrigger) & (shipmentstobill['ServiceLevel'] == 'DRA')]

#bringing charge files back together
blt1 = pd.concat([dra,notdra]) 

#Consolidating values, i.e (ChargeDate, WHID, Contract #) 
    #chargedate and contract number for DRA only are different values than other servicelevels
    #OND shipments the whid is an NCT value, for other servicelevels we use wms whid
    
blt1['ChargeDate'] = np.where(blt1['ServiceLevel'] == 'DRA',blt1['GateOut'], blt1['ATD'])
blt1['WarehouseID'] = np.where(blt1['ServiceLevel'].isin(['OND','OaaS']),blt1['whid'],blt1['wmswhid'])
blt1['Contract'] = np.where(blt1['ServiceLevel']=='DRA',blt1['DrayContract'],blt1['OceanContract'])

#removing excess columns in big file
bltcolumns = ['SuID', 'ACISpoID', 'ChargeDate', 'WarehouseID', 'ShipmentID', 'ServiceLevel','ContainerNumber', 'country','CBM',
              'Supplier Currency','Contract','GateIn','CFS Exchange Rate', 'Ocean Exchange Rate',
             'Dray Exchange Rate','Some Charges']

blt1 = blt1[bltcolumns]
    
#Pivoting out charges; going from spo level granularity to charge type level granularity
blt = shipmentstobill[['ShipmentID','cfscharge','oceanbasecharge','ddoccharge','bunker', 'draybasecharge', 'drayfuelcharge', 
                       'psscharge', 'pierpasscharge','CranFee', 'Securitycharge', 'DTHCFee']] 

blt = pd.melt(blt, id_vars=['ShipmentID'],var_name = 'chargetype', value_name='UnitPrice').dropna().reset_index(drop=True)

#dropping lines w/o a charge
blt = blt[(blt['UnitPrice'] != 0)]

#merging shipment details to charges
blt1 = pd.merge(blt1,blt, on= 'ShipmentID', how='left') 

C:\Users\rl409b\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  
C:\Users\rl409b\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  # This is added back by InteractiveShellApp.init_path()


In [213]:
pd.set_option('display.max_colwidth', -1)
blt1['ChargeNote']  = blt1['chargetype'].map(NOTE_ROW_HEADER) 

blt1['ChargeNote'] = np.where(blt1['ContainerNumber'].isnull(),blt1['ChargeNote'],
                             blt1['ChargeNote'] + 'ContainerNumber: ' + blt1['ContainerNumber'])
blt1['ChargeNote'] = np.where(blt1['ServiceLevel'] == 'ACN',blt1['ChargeNote'] +', ACI: ' + blt1['ACISpoID'],
                              blt1['ChargeNote'])
blt1['ChargeNote'] = blt1['ChargeNote'] + ', Contract: ' + blt1['Contract'].fillna('')

In [214]:
#Writing External Note

cfscharges = ['cfscharge']
oceancharges = ['oceanbasecharge','ddoccharge','bunker','psscharge', 'pierpasscharge','Securitycharge', 'DTHCFee']
draycharges = ['draybasecharge', 'drayfuelcharge','CranFee']

blt1['ExchangeRate'] = np.where(blt1['chargetype'].isin(cfscharges),blt1['CFS Exchange Rate'],0)
blt1['ExchangeRate'] = np.where(blt1['chargetype'].isin(oceancharges),blt1['Ocean Exchange Rate'],blt1['ExchangeRate'])
blt1['ExchangeRate'] = np.where(blt1['chargetype'].isin(draycharges),blt1['Dray Exchange Rate'],blt1['ExchangeRate'])

blt1['OriginalChargeAmount'] = (blt1['UnitPrice'] / blt1['ExchangeRate']).round(1)

#Charge Dict
NOTE_ROW_HEADER = {
    'cfscharge': 'Asia Consolidation - CFS Charge, ',
    'oceanbasecharge': 'Ocean Freight - Base, ',
    'bunker':'Ocean Freight - Bunker Adjustment, ',
    'psscharge': 'Ocean Freigt - Peak Season Surchage, ',
    'ddoccharge': 'Ocean Freight - DDOC, ',
    'Securitycharge': 'Ocean Freight - Security Fee, ',
    'DTHCFee': 'Ocean Freight - DTHC, ',
    'CranFee': 'Drayage - Cranbury Gate Fee, ',
    'pierpasscharge': 'Drayage - Pier Pass Fee, ',
    'draybasecharge': 'Drayage - Base, ',
    'drayfuelcharge': 'Drayage - Fuel, ',
    'InsuranceCharge': 'CargoProtect Insurance '    
    
}

#externalnote functions
def generate_note(s):
    try:
        if s['ServiceLevel'] == 'ACN':
            return NOTE_ROW_HEADER[s['chargetype']] + generate_not_detailsAC(s)
        if s['ServiceLevel'] != 'ACN':
            return NOTE_ROW_HEADER[s['chargetype']] + generate_not_details(s)
    except KeyError:
        return f"Invalid Charge Type {s['chargetype']}"
    
def generate_not_detailsAC(s):
    return f"ContainerNumber: {s['ContainerNumber']}, ACI: {s['ACISpoID']}, Contract: {s['Contract']}"

def generate_not_details(s):
    return f"ContainerNumber: {s['ContainerNumber']}, Contract: {s['Contract']}"

#writing ExternalNote W/O Original Rate    
blt1['ExternalNote'] = blt1.apply(generate_note, axis = 1)

def currencyconversionnote(s):
    if s['ExchangeRate'] != 1:
        return s['ExternalNote'] + ', Converted From: ' + str(s['OriginalChargeAmount'])
    else:
        return s['ExternalNote']
    
#Adding OriginalRate   
blt1['ExternalNote'] = blt1.apply(currencyconversionnote, axis = 1)   

In [215]:
#final BLT formatting

blt2 = blt1

#addding random columns for blt
blt2['QTY'] = 1
bltextracol = ['FulfillmentPoNumber', 'SprID','StartDate','EndDate','VatType']
blt2 = blt2.reindex(columns=blt2.columns.tolist() + bltextracol) 
blt2[bltextracol] = ''

#trimmingcolumns -- keeping chargetype, containernumber, somecharges, and cbm for data verifcation purposes 
blt5 = blt2[['SuID', 'ChargeDate', 'QTY', 'UnitPrice','UnitPrice', 'WarehouseID', 'ShipmentID', 'FulfillmentPoNumber', 'SprID',#
             'StartDate','EndDate','ExternalNote','VatType','ServiceLevel','chargetype', 'ContainerNumber', 'country',
             'CBM','Supplier Currency','Some Charges']]

#renaming columns for blt
finalbltcolumns = ['SupplierID', 'ChargeDate', 'QTY', 'UnitPrice', 'TotalAmount', 'WarehouseID','SpoID', 'FulfillmentPoNumber',
             'SprID', 'StartDate', 'EndDate', 'ExternalNote','VatType','ServiceLevel','chargetype','ContainerNumber',
              'country', 'CBM','Supplier Currency','Some Charges']

blt5.columns = [finalbltcolumns]

#formatting dtypes for blt 
bltnumbers = ['SupplierID','WarehouseID']
blt5[bltnumbers] = blt5[bltnumbers].astype(int).round(2)

#dropping any dupes
finalblt = blt5.drop_duplicates().reset_index(drop=True)

#outputing to T-Drive
path = (r'T:\SalesAndOps\Logistics\International Supply Chain\Billing and Reconciliation\Flexible Billing\Charge Output')

output_file1 = os.path.join(path,str(today)+' - AllCharges.csv')
output_file2 = os.path.join(path,str(today)+' - ShipmentsToBill.csv')
output_file3 = os.path.join(path,str(today)+' - Bulk Upload.csv')

finalblt.to_csv(output_file3)
allcharges.to_csv(output_file1)
shipmentstobill.to_csv(output_file2)

In [216]:
#supplier specific logic below 

#filltering charges to only those being sent to a supplier
firstbiz = pd.date_range('9/1/2020', '12/1/2022', freq='BMS')

## dictionary with suppliers in program

SUPPLIER_PREFERENCE_DICT = {
    #milestone options are 1strecipt or atd, gate-in
    #cadence options are not null or monthly
    
    12128: {
            'milestone': '1streceipt',
            'cadence': 'notnull',
        },
    1837: {
        'milestone': 'ATD',
        'cadence': 'monthly',
    }
}


##adding values from dictionary

def milestonedate (df):
    try: 
        supplier_info = SUPPLIER_PREFERENCE_DICT[df['SuID']] 
        milestone_date = df[supplier_info['milestone']] 
        return milestone_date
    except: ''
        
def cadence (df):
    try: 
        cadence = SUPPLIER_PREFERENCE_DICT[df['SuID']]['cadence'] 
        return cadence
    except: ''

allcharges['milestonedate'] = allcharges.apply(milestonedate, axis =1)
allcharges['cadence'] = allcharges.apply(cadence, axis =1)

#filtering df to only shipments with requiste milestone
supplieroutput = allcharges[(allcharges['milestonedate'].notnull())].reset_index(drop=True)

#spliting df into monthly, not null 
monthlycharges = supplieroutput[(supplieroutput['cadence']=='monthly')]
notnullcharges = supplieroutput[(supplieroutput['cadence']=='notnull')]
nntodaycharges = supplieroutput[(supplieroutput['milestonedate'] == today)]

#monthly workflow
month = today.month - 1 
month_mask = supplieroutput['milestonedate'].map(lambda x: x.month) == month ##only filtering on month
month_output = supplieroutput[month_mask]
monthlysuids = month_output['SuID'].unique()

C:\Users\rl409b\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.


In [217]:
testtoday =  '2020-9-01'
#outputing to shared drive
DataFrameDict = {elem : pd.DataFrame for elem in monthlysuids}
if testtoday in firstbiz:
    for key in DataFrameDict:
        month_output[:][month_output.SuID == key].to_csv(os.path.join(path,str(today) + ', SUID-' + str(key) +'.csv'))

nn_suids = notnullcharges['SuID'].unique()
nn_DataFrameDict = {elem : pd.DataFrame for elem in nn_suids}
for key in nn_DataFrameDict:
   nntodaycharges[:][nntodaycharges.suid == key].to_csv((os.path.join(path,str(today) + ', SUID-' + str(key) +'.csv')))